In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("../input/sunspots/Sunspots.csv")

data.head()

In [ ]:
#마지막 거 빼고는 쓸모가 없음
#We need only the last column
data = data["Monthly Mean Total Sunspot Number"]

#Normalization
data = data / max(data)

#to numpy
data = np.array(data)
print(data[:5])
print(data.shape) #3265,

In [ ]:
time_split = 3000
window_size = 30

train = data[:3000]
test_x = data[3000:]
print(len(test_x))

'''
train을 window_size만큼 잘라서 데이터셋 형성
slice data

if) window_size = 60

train_x = data[0:59]  data[1:60]   ...  data[2540:2599]
train_y = data[60]    data[61]     ...  data[3000]
'''

train_x = []
train_y = []
for i in range(len(train) - window_size - 1) :
    tmp_x = train[i : i+window_size]
    tmp_y = train[i+window_size]
    train_x.append(tmp_x)
    train_y.append(tmp_y)

train_x = np.array(train_x)
train_x = train_x.reshape((-1, window_size, 1))
train_y = np.array(train_y)
train_y = train_y.reshape((-1, 1))
print(train_x.shape, train_y.shape)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import Callback


model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(128, return_sequences=False)))
model.add(Dense(32, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation = 'linear'))

optimizer = Adam(lr = 1e-3)
model.compile(loss = Huber(), optimizer = optimizer, metrics = ["mae"])

with tf.device('/gpu:0'):
    model.fit(train_x, train_y, batch_size = 256, epochs = 100, verbose = 2)

In [ ]:
import matplotlib.pyplot as plt

#테스트를 위한 데이터셋 만들기
#make data for test (window_size)
test_data = []
train_y = train_y.reshape((-1))
for i in range(window_size) :
    tmp_data = train_y[-window_size*2 + i : -window_size + i]
    tmp_data = np.reshape(tmp_data, (1, window_size, 1))
    predict = model.predict(tmp_data)
    predict = predict.reshape((1))
    test_data.append(predict)

#predict
predict_list = []
for i in range(len(test_x)) :
    test_data = np.array(test_data).reshape((1, window_size, 1))
    predict = model.predict(test_data)
    predict = predict.reshape((1))
    test_data = test_data.reshape((window_size))
    predict_list.append(predict)
    test_data = np.append(test_data[1:], predict)

predict_list = np.array(predict_list)
predict_list = predict_list.reshape((-1))
times = list(range(len(data)))
plt.plot(times, data)
plt.plot(times[-len(test_x):], predict_list)
plt.show()

plt.plot(times[-len(test_x):], test_x)
plt.plot(times[-len(test_x):], predict_list)
plt.show()